# COMP5318 - Machine Learning and Data Mining 

## Tutorial 11 - Neural Networks & Deep Learning

**Semester 2, 2019**

**Objectives:**

* To learn about Perceptron
* To learn about dimensionality reduction 
* To learn about simple DNN 2 layers
* To learn about CNN
* To learn about Dropout, Pooling

**Instructions:**
* Exercises to be completed on IPython notebook such as: 
   * Ipython 3 (Jupyter) notebook installed on your computer http://jupyter.org/install (you need to have Python installed first https://docs.python.org/3/using/index.html )
   * Web-based Ipython notebooks such as Google Colaboratory https://colab.research.google.com/ 
   
* If you are using Jupyter intalled on your computer, Go to File->Open. Drag and drop "lab11.ipynb" file to the home interface and click upload. 
* If you are using Google Colaboratory, Click File->Upload notebook, and and upload "lab11.ipynb" file
* Complete exercises in "lab10.ipynb".
* To run the cell you can press Ctrl-Enter or hit the Play button at the top.
* Complete all exercises marked with **TODO**.
* Save your file when you are done with the exercises, so you can show your tutor next week.


Lecturers: Nguyen Hoang Tran

Tutors: Fengxiang He, Shaojun Zhang, Fangzhou Shi, Yang Lin, Iwan Budiman, Zhiyi Wang, Canh Dinh, Yixuan Zhang, Rui Dong, Haoyu He, Dai Hoang Tran, Peibo Duan

## 1. Perceptron

In [ ]:
import numpy as np
import matplotlib.pylab as pl
import math
%matplotlib inline

Loading data

In [ ]:
from __future__ import print_function 
import numpy as np 
import matplotlib.pyplot as plt
np.random.seed(2)

means = [[-1, 0], [1, 0]]
cov = [[.3, .2], [.2, .3]]
N = 10
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)

X = np.concatenate((X0, X1), axis = 0)
y = np.concatenate((np.ones(N), -1*np.ones(N)))

#fig,axs=plt.subplots(nrows,ncols,figsize=(width,height))
plt.plot(X0[:, 0], X0[:, 1], 'bs', markersize = 8, alpha = .8)
plt.plot(X1[:, 0], X1[:, 1], 'ro', markersize = 8, alpha = .8)
plt.show()

This data set has 2 classes in total. We will use perceptron algorithm to make a decision boundary for this dataset.

In perception, we minimizes the number of errors on the training dataset follows:

$$ \epsilon = \sum_n 1_{[y_n \neq sign(w^tx_n)]}$$

**Algorithm:**

For a randomly chosen data point $(x_n, y_n)$ make small changes to $w$ so that : $y_n = sign(w^tx_n)$:

2 case:
- If $y_n = sign(w^tx_n)$: Do nothing
- If $y_n \neq sign(w^tx_n)$: update w : $w_{t+1} = w_{t} + y_nx_n$ => we only update $w$ for misclassified point. The update process will be stopped when there is no misclassified point.

**Create Perception Algorithm from scratch:**

In [ ]:
def predict(w, X):    
    '''
    predict label of each row of X, given w 
    '''
    return np.sign(X.dot(w))

def perceptron(X, y, w_init):
    w = w_init
    w_hist = [w]
    mis_points = [] # list of misclassified points
    while True:
        pred = predict(w, X)
        mis_idxs = # TODO: Get all misclassified points
        num_mis = mis_idxs.shape[0]
        if num_mis == 0:
            return (w_hist, mis_points)
        # random pick one misclassified point 
        random_id = np.random.choice(mis_idxs, 1)[0]
        mis_points.append(random_id)
        w = # TODO: Update weight vector
        w_hist.append(w)

Make the prediction

In [ ]:
np.random.seed(73)
Xbar = np.concatenate((np.ones((2*N, 1)), X), axis = 1)
d = Xbar.shape[1]
w_init = np.random.randn(d)
w_hist, m = perceptron(Xbar, y, w_init)

Visualize the change of boundary during the learning processs depends on number of update 

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages    
def draw_line(plt, w):
    w0, w1, w2 = w[0], w[1], w[2]
    if w2 != 0:
        x11, x12 = -100, 100
        return plt.plot([x11, x12], [-(w1*x11 + w0)/w2, -(w1*x12 + w0)/w2], 'k')
    else:
        x10 = -w0/w1
        return plt.plot([x10, x10], [-100, 100], 'k')

In [ ]:
ncols = m 
if(len(m) >= 3):
    ncols = 3
nrows = math.ceil(len(m)/3)
width = 4.5*ncols 
height = 3.5*nrows

filename = 'pla_visualize1.pdf'
with PdfPages(filename) as pdf: 
    plt.close('all')
    fig,axs=plt.subplots(nrows,ncols,figsize=(width,height))
    ids = range(len(m)+1)
    for i, k in enumerate(ids[1:]):
        #print("i,k",i,k)
        r = i//ncols 
        c = i%ncols 
        str0 = 'iter {}/{}'.format(i+1, len(ids)-1)
        if(nrows > 1):
            temp = axs[r, c]
        else:
            temp = axs[c]
        if nrows > 1:
            temp.set_title(str0)
            temp.plot(X0[:, 0], X0[:, 1], 'bs', markersize = 8, alpha = .8)
            temp.plot(X1[:, 0], X1[:, 1], 'ro', markersize = 8, alpha = .8)
            temp.plot(0, 0, 'k.', markersize = 8, alpha = .8)
            temp.axis([0 , 6, -2, 4])
            draw_line(temp, w_hist[k])
            wx, wy = w_hist[k][1], w_hist[k][2]
            temp.annotate('', xy=(wx, wy), xytext=(0, 0),arrowprops=dict(arrowstyle="simple", connectionstyle="arc3", ec = 'green', fc = 'green'))
        if k < nrows*ncols:
            
            # get misclassified point
            xmis = X[m[k], 0] 
            ymis = X[m[k], 1]
            
            #circle around the misclassified point
            circle = plt.Circle((xmis, ymis), 0.2, color='k', fill = False)
            temp.add_artist(circle)
            
            #vector to xmis
            temp.annotate('', xy=(xmis, ymis), xytext=(0, 0),
                        arrowprops=dict(arrowstyle="simple", connectionstyle="arc3", ec = 'orange', fc = 'orange'))

            if m[k] > 10:
                #New w if the misclassified point is in red 
                temp.annotate('', xytext=(0, 0), xy=(wx - xmis, wy-ymis),
                        arrowprops=dict(arrowstyle="simple", connectionstyle="arc3", ec = 'red', fc = 'red'))
            else: # the misclassified point is in blue
                temp.annotate('', xytext=(0, 0), xy=(wx + xmis, wy+ymis),
                        arrowprops=dict(arrowstyle="simple", connectionstyle="arc3", ec = 'red', fc = 'red'))
                
        temp.axis('scaled')
        temp.axis([-3, 3, -1.5, 2.5])
    pdf.savefig(bbox_inches = 'tight')
    plt.show()

Blue points: Class 1, Red points: Class -1


**Green Vector** is $w_t$.

**Red Vertor** is $w_{t+1}$

The circled points are misclassified point ($x_i$)

**Orange Vector** is xi

- If $y_i = 1$ (blue), red vector = Sum(Green Vector, Orange Vector)
- If $y_i = −1$, red vector = Sub(Green Vector, Orange Vector)

## 2. Simple autoencoder
The goal here is to generate a representation of our data with lower dimension. To do so, autoencoders first transform the data to a low-dimension representation using an encoder network, and then transform the low-dimension representation back to the original space using a decoder network.

Here we start with a single fully-connected neural layer as encoder and as decoder. We will be encoding MNIST digit images (dim=784) into a space of dimension 32, hence the compression factor is 784/32 = 24.5.

In this part we use CNN in classify MNIST dataset.

## Loading MNIST data from Keras
We use one of the dataset included in Keras: MNIST (https://en.wikipedia.org/wiki/MNIST_database).

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_val = x_test[:5000,:]
x_test = x_test[5000:,:]
y_val = y_test[:5000]
y_test = y_test[5000:]

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# this is our input placeholder
input_img = Input(shape=(784,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
decoded = Dense(784, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# Reshape data to fit the autoencoder layout and normalize it
x_train_ae = x_train.astype('float32') / 255.
x_test_ae = x_test.astype('float32') / 255.
x_val_ae = x_val.astype('float32') / 255.
x_train_ae = x_train_ae.reshape((len(x_train_ae), np.prod(x_train_ae.shape[1:])))
x_test_ae = x_test_ae.reshape((len(x_test_ae), np.prod(x_test_ae.shape[1:])))
x_val_ae = x_val_ae.reshape((len(x_val_ae), np.prod(x_val_ae.shape[1:])))

Train the autoencoder to reconstruct MNIST digits.

In [ ]:
# Configure the model to use a per-pixel binary crossentropy loss, and Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Train the model for 10 epochs
autoencoder.fit(x_train_ae, x_train_ae, epochs=10, batch_size=256,
                shuffle=True, validation_data=(x_val_ae, x_val_ae))

Vizualize the reconstructed inputs and the encoded representations.

In [ ]:
# Retrieve the encoder and decoder as separate networks
encoder = Model(input_img, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

encoded_imgs = encoder.predict(x_test_ae)
decoded_imgs = decoder.predict(encoded_imgs)

# use Matplotlib
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test_ae[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

Note that the autoencoder is not fully converged yet as it was trained on only 10 epochs. You may want to try and run it for longer to get better digit reconstruction.

## 3. DNN 

In this section, we will use a simple DNN with 2 layers to classify MNIST dataset and then we will compare its performance with CNN in the next section.

Before building DNN network, we need to load MNIST data and preprocess data

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
num_train = 6000 # Max is 60000
num_test = 1000 # Max is 5000
num_val = 1000 # Max is 5000
# Only keep a subset of the data
x_train_nn = x_train[:num_train,:,:,None]
y_train_nn = y_train[:num_train]
x_test_nn = x_test[:num_test,:,:,None]
y_test_nn = y_test[:num_test]
x_val_nn = x_test[:num_val,:,:,None]
y_val_nn = y_test[:num_val]

x_train_nn = x_train_nn.astype('float32')
x_test_nn = x_test_nn.astype('float32')
x_val_nn = x_val_nn.astype('float32')
x_train_nn /= 255.
x_test_nn /= 255.
x_val_nn /= 255.
# Convert class vectors to binary class matrices using keras.utils.to_categorical function 
y_train_nn = # TODO
y_test_nn =  # TODO
y_val_nn =  # TODO

In [ ]:
x_train_dnn = x_train_nn.reshape(6000,784)
x_test_dnn = x_test_nn.reshape(1000,784)
x_val_dnn = x_val_nn.reshape(1000,784)

**Build a simple 2 layer DNN**

In [ ]:
from keras.layers import Dense # Dense layers are "fully connected" layers
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/

model = Sequential()
model.add(Dense(units=256, activation='sigmoid', input_shape=x_train_dnn.shape[1:]))
# TODO: add the last layer as softmax activation 
model.add(# TODO)
model.summary()

In [ ]:
epochs = 10
model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])

history_dnn = model.fit(x_train_dnn, y_train_nn, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val_dnn, y_val_nn), shuffle=True)

plt.plot(history_dnn.history['acc'])
plt.plot(history_dnn.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show()

## 4. CNN

#### Simple CNN

In [ ]:
# Create CNN topology
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train_nn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print(model.summary())

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit model to data
epochs = 3
print(epochs)
history_cnn1 = model.fit(x_train_nn, y_train_nn, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val_nn, y_val_nn), shuffle=True)
plt.plot(history_cnn1.history['acc'])
plt.plot(history_cnn1.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show()

In [ ]:
score = model.evaluate(x_test_nn, y_test_nn, batch_size=32, verbose=1)
print("\nTest accuracy is {}%".format(100.0*score[1]))

CNN gives a better results than DNN but it takes a significant long time to train the model.

## 5. Adding dropout and pooling to the CNN
Max pooling and dropouts are tricks that improve deep neural networks.

Max pooling (http://yann.lecun.com/exdb/publis/pdf/boureau-icml-10.pdf) speeds up CNN training and encourages CNN to learn translation-invariant features.
Dropout (https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) is a regularisation technique preventing the model from overfitting the data.

In [ ]:
# Create CNN topology
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train_nn.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(# TODO: add Pooling layer size (2,2))
model.add(# TODO: add drop out layer, change the value of dropout, and discuss the differences)

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(# TODO: add Pooling layer size (2,2))
model.add(# TODO: add drop out layer, change the value of dropout, and discuss the differences)

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(# TODO: add drop out layer, change the value of dropout, and discuss the differences)
model.add(Dense(num_classes))
model.add(Activation('softmax'))
print(model.summary())

In [ ]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit model to data
epochs = 3
history_cnn2 = model.fit(x_train_nn, y_train_nn, batch_size=batch_size, epochs=epochs,
              validation_data=(x_val_nn, y_val_nn), shuffle=True)

In [ ]:
plt.plot(history_cnn2.history['acc'])
plt.plot(history_cnn2.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='best')
plt.show()

Evaluate model score on test set.

In [ ]:
score = model.evaluate(x_test_nn, y_test_nn, batch_size=32, verbose=1)
print("\nTest accuracy is {}%".format(100.0*score[1]))